In [2]:
import quacc
import ase
from quacc import SETTINGS
import os

In [14]:
from quacc.recipes.dftb.core import relax_job
from jobflow import job, Flow, run_locally
from ase.io import read


def relaxation_dftb_plus(atoms):

    return relax_job(
        atoms=atoms,
        method="GFN2-xTB",
        Driver_="GeometryOptimization",
        Driver_LatticeOpt="Yes",
        Driver_MaxForceComponent="1e-6",
        Driver_MaxSteps=100,
        Driver_OutputPrefix="aaa_end", # output file prefix
        Driver_AppendGeometries="Yes",
        Hamiltonian_SCC='Yes',
        Hamiltonian_SCCTolerance='1e-2',
        Hamiltonian_MaxSccIterations=3000,
    )

atoms = read('NaCl_noise.cif')
relaxation = relaxation_dftb_plus(atoms)
result = run_locally(relaxation)
print(result[list(result)[0]][1].output)
# flow = ([relaxation])


2024-05-07 11:28:55,166 INFO Started executing jobs locally
2024-05-07 11:28:55,168 INFO Starting job - relax_job (a7140e00-1805-4299-b86b-91db1c9cb8e4)


KeyboardInterrupt: 

In [43]:
from ase.io import read, write
from quacc.recipes.dftb.core import relax_job as dftb_relax_job
from quacc.recipes.gulp.core import relax_job as gulp_relax_job
from jobflow import job, Flow, run_locally
import gzip
import shutil

@job
def relaxation_dftb_plus(input_file):
    atoms = read(input_file)
    if 'spacegroup' in atoms.info:
        del atoms.info['spacegroup'] 

    result = dftb_relax_job(
        atoms=atoms,
        method="GFN2-xTB",
        Driver_="GeometryOptimization",
        Driver_LatticeOpt="Yes",
        Driver_MaxForceComponent="1e-6",
        Driver_MaxSteps=100,
        Driver_OutputPrefix="aaa_end",
        Driver_AppendGeometries="Yes",
        Hamiltonian_SCC='Yes',
        Hamiltonian_SCCTolerance='1e-4',
        Hamiltonian_MaxSccIterations=3000,
    )
    return result

@job
def conversion_job(gen_gz_file_directory, gen_gz_file_name, output_file_name):
    gen_gz_file = f'{gen_gz_file_directory}/{gen_gz_file_name}.gen.gz'
    output_cif_file = f'{gen_gz_file_directory}/{output_file_name}.cif'
    
    gen_file = gen_gz_file[:-3]  # Removes the .gz extension
    
    with gzip.open(gen_gz_file, 'rb') as f_in:
        with open(gen_file, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    
    atoms = read(gen_file)
    write(output_cif_file, atoms)
    
    return f"Converted {gen_gz_file} to {output_cif_file}"

@job
def relaxation_gfnff(input_file):
    atoms = read(input_file)
    result = gulp_relax_job(
        atoms=atoms,
        use_gfnff=True,
        relax_cell=True,
        keywords={

        }
    )
    return result["atoms"]

input_file = '/Users/ruben_mac/Desktop/project/NaCl_noise.cif'
gen_gz_file_directory = '/Users/ruben_mac/Desktop/project'
gen_gz_file_name = 'aaa_end'
output_file_name = 'aaa_end'

flow = Flow([
    relaxation_dftb_plus(input_file),
    conversion_job(gen_gz_file_directory, gen_gz_file_name, output_file_name),
    relaxation_gfnff(input_file)
])

results = run_locally(flow)


2024-05-07 10:22:39,492 INFO Started executing jobs locally


INFO:jobflow.managers.local:Started executing jobs locally


2024-05-07 10:22:39,494 INFO Starting job - relaxation_dftb_plus (923197a1-bfd7-4fa5-a629-a9ecb8535072)


/Users/ruben_mac/miniconda3/envs/test/lib/python3.9/site-packages/jobflow/core/flow.py:423: UserWarning: Some jobs are not connected, their ordering may be random
  for node in itergraph(graph):
INFO:jobflow.core.job:Starting job - relaxation_dftb_plus (923197a1-bfd7-4fa5-a629-a9ecb8535072)
INFO:quacc.runners.prep:Calculation will run at /Users/ruben_mac/Desktop/Project/tmp-quacc-2024-05-07-17-22-39-498110-27066


2024-05-07 10:22:39,540 INFO relaxation_dftb_plus failed with exception:
Traceback (most recent call last):
  File "/Users/ruben_mac/miniconda3/envs/test/lib/python3.9/site-packages/jobflow/managers/local.py", line 114, in _run_job
    response = job.run(store=store)
  File "/Users/ruben_mac/miniconda3/envs/test/lib/python3.9/site-packages/jobflow/core/job.py", line 583, in run
    response = function(*self.function_args, **self.function_kwargs)
  File "/var/folders/kf/59l21j410zndl1bxmzhg6p_r0000gn/T/ipykernel_11365/4222207419.py", line 14, in relaxation_dftb_plus
    result = dftb_relax_job(
  File "/Users/ruben_mac/miniconda3/envs/test/lib/python3.9/site-packages/quacc/recipes/dftb/core.py", line 118, in relax_job
    return run_and_summarize(
  File "/Users/ruben_mac/miniconda3/envs/test/lib/python3.9/site-packages/quacc/recipes/dftb/_base.py", line 59, in run_and_summarize
    final_atoms = run_calc(atoms, geom_file=GEOM_FILE, copy_files=copy_files)
  File "/Users/ruben_mac/minico

...

INFO:jobflow.core.job:Starting job - relaxation_gfnff (c4b1c7f1-8af8-4d41-8459-e5b67afddb4c)
INFO:quacc.runners.prep:Calculation will run at /Users/ruben_mac/Desktop/Project/tmp-quacc-2024-05-07-17-22-39-549805-96801


2024-05-07 10:22:39,591 INFO relaxation_gfnff failed with exception:
Traceback (most recent call last):
  File "/Users/ruben_mac/miniconda3/envs/test/lib/python3.9/site-packages/jobflow/managers/local.py", line 114, in _run_job
    response = job.run(store=store)
  File "/Users/ruben_mac/miniconda3/envs/test/lib/python3.9/site-packages/jobflow/core/job.py", line 583, in run
    response = function(*self.function_args, **self.function_kwargs)
  File "/var/folders/kf/59l21j410zndl1bxmzhg6p_r0000gn/T/ipykernel_11365/4222207419.py", line 48, in relaxation_gfnff
    result = gulp_relax_job(
  File "/Users/ruben_mac/miniconda3/envs/test/lib/python3.9/site-packages/quacc/recipes/gulp/core.py", line 115, in relax_job
    return run_and_summarize(
  File "/Users/ruben_mac/miniconda3/envs/test/lib/python3.9/site-packages/quacc/recipes/gulp/_base.py", line 104, in run_and_summarize
    final_atoms = run_calc(
  File "/Users/ruben_mac/miniconda3/envs/test/lib/python3.9/site-packages/quacc/runners/

/bin/sh: gulp: command not found
INFO:jobflow.managers.local:relaxation_gfnff failed with exception:
Traceback (most recent call last):
  File "/Users/ruben_mac/miniconda3/envs/test/lib/python3.9/site-packages/jobflow/managers/local.py", line 114, in _run_job
    response = job.run(store=store)
  File "/Users/ruben_mac/miniconda3/envs/test/lib/python3.9/site-packages/jobflow/core/job.py", line 583, in run
    response = function(*self.function_args, **self.function_kwargs)
  File "/var/folders/kf/59l21j410zndl1bxmzhg6p_r0000gn/T/ipykernel_11365/4222207419.py", line 48, in relaxation_gfnff
    result = gulp_relax_job(
  File "/Users/ruben_mac/miniconda3/envs/test/lib/python3.9/site-packages/quacc/recipes/gulp/core.py", line 115, in relax_job
    return run_and_summarize(
  File "/Users/ruben_mac/miniconda3/envs/test/lib/python3.9/site-packages/quacc/recipes/gulp/_base.py", line 104, in run_and_summarize
    final_atoms = run_calc(
  File "/Users/ruben_mac/miniconda3/envs/test/lib/python

2024-05-07 10:22:39,592 INFO Finished executing jobs locally


INFO:jobflow.managers.local:Finished executing jobs locally
